In [ ]:
using InstantiateFromURL
# optionally add arguments to force installation: instantiate = true, precompile = true
github_project("mmikhasenko/Introduction2AmplitudeAnalysis", version = "0.0.1")

In [ ]:
using WebIO
WebIO.install_jupyter_nbextension()

In [50]:
using Plots
theme(:wong)

In [51]:
using Interact

In [ ]:
using Parameters

In [1]:
λ(x,y,z) = x^2+y^2+z^2-2x*y-2y*z-2z*x

λ (generic function with 1 method)

In [27]:
@with_kw struct ThreeBodySystem
    m0::Float64
    m1::Float64
    m2::Float64
    m3::Float64
end
sumsq(tbs::ThreeBodySystem) = tbs.m0^2+tbs.m1^2+tbs.m2^2+tbs.m3^2

sumsq (generic function with 1 method)

In [28]:
tbs = ThreeBodySystem(m0=1.1, m1=0.3, m2=0.3, m3=0.3)

ThreeBodySystem
  m0: Float64 1.1
  m1: Float64 0.3
  m2: Float64 0.3
  m3: Float64 0.3


In [31]:
@with_kw struct Invariants
    σ1::Float64
    σ2::Float64
    σ3::Float64
end
function Invariants(tbs::ThreeBodySystem;σ1=-1.0,σ2=-1.0,σ3=-1.0)
    sign(σ1)+sign(σ2)+sign(σ3)!=1 && error("the method works with TWO invariants given")
    σ3 < 0 && return Invariants(σ1=σ1,σ2=σ2,σ3=sumsq(tbs)-σ1-σ2)
    σ1 < 0 && return Invariants(σ2=σ2,σ3=σ3,σ1=sumsq(tbs)-σ2-σ3)
    σ2 < 0 && return Invariants(σ3=σ3,σ1=σ1,σ2=sumsq(tbs)-σ3-σ1)
end

Invariants

In [32]:
Invariants(tbs,σ1=0.7^2,σ2=0.7^2)

Invariants
  σ1: Float64 0.48999999999999994
  σ2: Float64 0.48999999999999994
  σ3: Float64 0.5000000000000004


In [132]:
p1mod(σs::Invariants,tbs::ThreeBodySystem) = sqrt(λ(tbs.m0^2,tbs.m1^2,σs.σ1))/(2tbs.m0)
p2mod(σs::Invariants,tbs::ThreeBodySystem) = sqrt(λ(tbs.m0^2,tbs.m2^2,σs.σ2))/(2tbs.m0)
p3mod(σs::Invariants,tbs::ThreeBodySystem) = sqrt(λ(tbs.m0^2,tbs.m3^2,σs.σ3))/(2tbs.m0)
#
cosθh12(σs::Invariants,tbs::ThreeBodySystem) =
    (2tbs.m0^2*(tbs.m1^2+tbs.m2^2-σs.σ3)+(tbs.m0^2+tbs.m1^2-σs.σ1)*(tbs.m0^2+tbs.m2^2-σs.σ2)) /
        sqrt(λ(tbs.m0^2,tbs.m1^2,σs.σ1)*λ(tbs.m0^2,tbs.m2^2,σs.σ2))
cosθh23(σs::Invariants,tbs::ThreeBodySystem) =
    (2tbs.m0^2*(tbs.m2^2+tbs.m3^2-σs.σ1)+(tbs.m0^2+tbs.m2^2-σs.σ2)*(tbs.m0^2+tbs.m3^2-σs.σ3)) /
        sqrt(λ(tbs.m0^2,tbs.m2^2,σs.σ2)*λ(tbs.m0^2,tbs.m3^2,σs.σ3))
cosθh31(σs::Invariants,tbs::ThreeBodySystem) =
    (2tbs.m0^2*(tbs.m3^2+tbs.m1^2-σs.σ2)+(tbs.m0^2+tbs.m3^2-σs.σ3)*(tbs.m0^2+tbs.m1^2-σs.σ1)) /
        sqrt(λ(tbs.m0^2,tbs.m3^2,σs.σ3)*λ(tbs.m0^2,tbs.m1^2,σs.σ1))
#
Kibble(σs::Invariants,tbs::ThreeBodySystem) = λ(λ(tbs.m0^2,tbs.m1^2,σs.σ1),
                                                λ(tbs.m0^2,tbs.m2^2,σs.σ2),
                                                λ(tbs.m0^2,tbs.m3^2,σs.σ3))
#
σ2of1(z, σ1; tbs::ThreeBodySystem) = tbs.m3^2 + tbs.m1^2 +
    (σ1+tbs.m3^2-tbs.m2^2)*(tbs.m0^2-tbs.m1^2-σ1)/(2σ1) -
    sqrt(λ(tbs.m0^2,tbs.m1^2,σ1)*λ(tbs.m2^2,tbs.m3^2,σ1))/(2σ1)*z

σ2of1 (generic function with 1 method)

In [140]:
@manipulate for σ1 in range((tbs.m2+tbs.m3)^2, (tbs.m0-tbs.m1)^2, length=500),
                σ2 in range((tbs.m3+tbs.m1)^2, (tbs.m0-tbs.m2)^2, length=500)
    σs = Invariants(tbs,σ1=σ1,σ2=σ2)
    (σs,α) = Kibble(σs,tbs) < 0 ? (σs,1.0) : (Invariants(tbs,σ1=0.7^2,σ2=0.7^2), 0.0)
    #
    plot(layout=grid(1,2), size=(900,350))
    plot!(sp=2, xaxis=false, yaxis=false, aspect_ratio=1)
    #
    _p1mod = p1mod(σs,tbs)
    plot!(sp=2, [0,-_p1mod], [0, 0], lab="", arrow=true, lw=2, α=α)
    #
    _p2mod = p2mod(σs,tbs)
    _cosθh12 = cosθh12(σs,tbs)
    _sinθh12 = sqrt(1-_cosθh12^2)
    plot!(sp=2, [0,-_p2mod*_cosθh12], [0,_p2mod*_sinθh12], lab="", arrow=true, lw=2, α=α, seriescolor=2)
    plot!(sp=2, [0,-_p2mod*_cosθh12].-_p1mod, [0,_p2mod*_sinθh12], lab="", arrow=true, lw=0.5, ls=:dash, α=α, seriescolor=2)
    #
    _p3mod = p3mod(σs,tbs)
    _cosθh31 = cosθh31(σs,tbs)
    _sinθh31 = sqrt(1-_cosθh31^2)
    plot!(sp=2, [0,-_p3mod*_cosθh31], [0,-_p3mod*_sinθh31], lab="", arrow=true, lw=2, α=α, seriescolor=3)
    plot!(sp=2, [0,-_p3mod*_cosθh31].-_p1mod .-_p2mod*_cosθh12,
        [0,-_p3mod*_sinθh31].+_p2mod*_sinθh12, lab="", arrow=true, lw=0.5, ls=:dash, α=α, seriescolor=3)
    #
    scatter!(sp=1, [σ1], [σ2], lab="", ms=10,
            xlim=((tbs.m2+tbs.m3)^2, (tbs.m0-tbs.m1)^2),
            ylim=((tbs.m3+tbs.m1)^2, (tbs.m0-tbs.m2)^2))
    #
    σ1v = range((tbs.m2+tbs.m3)^2, (tbs.m0-tbs.m1)^2, length=100)
    σ2v⁺ = σ2of1.(1,σ1v; tbs=tbs)
    σ2v⁻ = σ2of1.(-1,σ1v; tbs=tbs)
    plot!(sp=1, σ1v, [σ2v⁺ σ2v⁻], lab="", l=(:gray))
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["σ1"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 500,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
250, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\mikha\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\mikha\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\mikha\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\mikha\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\mikha\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000014eda290, Task (runnable) @0x0000000014eda290), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.36\",\"0.360561\",\"0.361122\",\"0.361683\",\"0.362244\",\"0.362806\",\"0.363367\",\"0.363928\",\"0.364489\",\"0.36505\",\"0.365611\",\"0.366172\",\"0.366733\",\"0.367295\",\"0.367856